In [3]:
%pip install pandas
import pandas as pd

# Load the CSV content into a DataFrame
data = pd.read_csv('file/datos (4).csv')

# Display the first few rows of the DataFrame to understand its structure
data.head()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: c:\Users\user\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


,Fecha,Campania,Plataforma,inversion,evento_objetivo,Instalaciones,cvr
0,2025-01-02,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(GAMERS)_I...,Meta,58.12,9,28,0.321429
1,2025-01-06,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...,Meta,97.28,30,98,0.306122
2,2025-01-02,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...,Meta,93.51,15,90,0.166667
3,2025-01-03,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(GAMERS)_I...,Meta,59.91,15,30,0.500000
4,2025-01-04,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...,Meta,93.40,29,72,0.402778


In [4]:
# Convert 'Fecha' to datetime format for easier manipulation
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Find the most recent date in the dataset
most_recent_date = data['Fecha'].max()

# Determine the two periods for comparison
period_end = most_recent_date
period_start = period_end - pd.Timedelta(days=6)
previous_period_end = period_start - pd.Timedelta(days=1)
previous_period_start = previous_period_end - pd.Timedelta(days=6)

# Filter the data for the two periods
current_period_data = data[(data['Fecha'] >= period_start) & (data['Fecha'] <= period_end)]
previous_period_data = data[(data['Fecha'] >= previous_period_start) & (data['Fecha'] <= previous_period_end)]

# Remove platforms with zero or near-zero investment
current_period_data = current_period_data[current_period_data['inversion'] > 0.1]
previous_period_data = previous_period_data[previous_period_data['inversion'] > 0.1]

# Display the date ranges for the periods
period_start, period_end, previous_period_start, previous_period_end

(Timestamp('2025-01-06 00:00:00'),
 Timestamp('2025-01-12 00:00:00'),
 Timestamp('2024-12-30 00:00:00'),
 Timestamp('2025-01-05 00:00:00'))

In [5]:
# Group by platform and calculate the required metrics for each period
def calculate_metrics(data):
    metrics = data.groupby('Plataforma').agg({
        'evento_objetivo': 'sum',
        'inversion': 'sum',
        'Instalaciones': 'sum'
    }).reset_index()
    metrics['costo_medio_evento'] = metrics['inversion'] / metrics['evento_objetivo']
    metrics['costo_medio_instalacion'] = metrics['inversion'] / metrics['Instalaciones']
    metrics['cvr_medio'] = metrics['evento_objetivo'] / metrics['Instalaciones']
    return metrics

current_metrics = calculate_metrics(current_period_data)
previous_metrics = calculate_metrics(previous_period_data)

# Calculate percentage changes between the two periods
comparison = current_metrics.set_index('Plataforma').join(previous_metrics.set_index('Plataforma'), lsuffix='_current', rsuffix='_previous')
comparison['var_costo_medio_evento'] = (comparison['costo_medio_evento_current'] - comparison['costo_medio_evento_previous']) / comparison['costo_medio_evento_previous'] * 100
comparison['var_eventos'] = (comparison['evento_objetivo_current'] - comparison['evento_objetivo_previous']) / comparison['evento_objetivo_previous'] * 100
comparison['var_costo_medio_instalacion'] = (comparison['costo_medio_instalacion_current'] - comparison['costo_medio_instalacion_previous']) / comparison['costo_medio_instalacion_previous'] * 100
comparison['var_instalaciones'] = (comparison['Instalaciones_current'] - comparison['Instalaciones_previous']) / comparison['Instalaciones_previous'] * 100
comparison['var_inversion'] = (comparison['inversion_current'] - comparison['inversion_previous']) / comparison['inversion_previous'] * 100

# Display the comparison results
results = comparison[['var_costo_medio_evento', 'var_eventos', 'var_costo_medio_instalacion', 'var_instalaciones', 'var_inversion']]

results

,var_costo_medio_evento,var_eventos,var_costo_medio_instalacion,var_instalaciones,var_inversion
Plataforma,,,,,
Google Ads,-11.936126,4.785156,5.881718,-12.848158,-7.722132
Meta,-9.504950,6.607930,-11.263062,8.720113,-3.525101
TikTok Ads,-10.513276,-24.489796,5.664883,-36.051026,-32.428392
